# HR MCP Server Workshop
## Deploy an MCP Server on Amazon Bedrock AgentCore Runtime

### What you'll build
An HR MCP server with 5 tools (leave balance, leave requests, employee info, record updates, support tickets) deployed to AgentCore Runtime and connected to Amazon Quick.

### Architecture
```
Amazon Quick  ──OAuth──▶  Cognito  ──JWT──▶  AgentCore Runtime  ──▶  HR MCP Server
  (client_credentials)       (domain +           (validates JWT)        (5 HR tools)
                              scope +
                              secret)
```

**⚠️ Do NOT use "Run All Cells"** — Step 4 (deploy) takes 3-5 minutes.

### Prerequisites
- SageMaker JupyterLab with an execution role that has `AdministratorAccess`

---
## Step 1: Install Dependencies
⏱️ ~2 minutes

- `bedrock-agentcore-starter-toolkit` — CLI to configure and deploy to AgentCore Runtime
- `fastmcp` / `mcp` — MCP server and client libraries
- `zip` — system utility required by direct code deploy (zips your code for upload)

In [ ]:
!pip install -q fastmcp boto3 mcp httpx pyyaml bedrock-agentcore-starter-toolkit==0.2.10
!apt-get update -qq && apt-get install -y -qq zip 2>/dev/null
print('\n✅ All dependencies installed')

---
## Step 2: Set Up Cognito Authentication
⏱️ ~10 seconds

Creates everything needed for JWT auth between Quick client and AgentCore:

| Resource | Why |
|----------|-----|
| **User Pool** | Container for users and OAuth config |
| **Cognito Domain** | Creates the `/oauth2/token` endpoint URL that Quick client calls |
| **Resource Server + Scope** | `client_credentials` grant requires at least one scope (`hr-mcp/access`) |
| **App Client with Secret** | Quick client sends `client_id` + `client_secret` to get a JWT token |
| **Test User** | For testing auth directly via username/password |

In [ ]:
import boto3
import json
import yaml
import time
from utils import setup_cognito_user_pool

AWS_REGION = 'us-east-1'

cognito_config = setup_cognito_user_pool()

print(f'\n✓ Pool ID:        {cognito_config["pool_id"]}')
print(f'✓ Client ID:      {cognito_config["client_id"]}')
print(f'✓ Client Secret:  {cognito_config["client_secret"][:10]}...')
print(f'✓ Token URL:      {cognito_config["token_url"]}')
print(f'✓ Discovery URL:  {cognito_config["discovery_url"]}')

---
## Step 3: Create AgentCore Execution Role
⏱️ ~5 seconds

AgentCore Runtime assumes this IAM role to run your MCP server code.

The trust policy must:
- Allow `bedrock-agentcore.amazonaws.com` to assume the role
- Include `SourceAccount` and `SourceArn` conditions (required by AgentCore)

In [ ]:
iam = boto3.client('iam', region_name=AWS_REGION)
sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']

role_name = 'AgentCore-HR-MCP-ExecutionRole'

trust_policy = {
    'Version': '2012-10-17',
    'Statement': [{
        'Sid': 'AssumeRolePolicy',
        'Effect': 'Allow',
        'Principal': {
            'Service': 'bedrock-agentcore.amazonaws.com'
        },
        'Action': 'sts:AssumeRole',
        'Condition': {
            'StringEquals': {
                'aws:SourceAccount': account_id
            },
            'ArnLike': {
                'aws:SourceArn': f'arn:aws:bedrock-agentcore:{AWS_REGION}:{account_id}:*'
            }
        }
    }]
}

try:
    resp = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description='Execution role for HR MCP Server on AgentCore Runtime'
    )
    execution_role_arn = resp['Role']['Arn']
    print(f'✓ Role created: {execution_role_arn}')
except iam.exceptions.EntityAlreadyExistsException:
    execution_role_arn = f'arn:aws:iam::{account_id}:role/{role_name}'
    iam.update_assume_role_policy(
        RoleName=role_name,
        PolicyDocument=json.dumps(trust_policy)
    )
    print(f'✓ Role exists, trust policy updated: {execution_role_arn}')

---
## Step 4: Configure and Deploy to AgentCore Runtime
⏱️ **3-5 minutes**

This cell does three things:

1. **`agentcore configure`** — creates `.bedrock_agentcore.yaml` (deployment manifest) with:
   - `direct_code_deploy` — zips your Python code, no Docker needed
   - `MCP` protocol — tells AgentCore this is an MCP server
   - Cognito JWT authorizer — validates tokens from Quick client

2. **30s wait** — IAM role needs time to propagate globally

3. **`agentcore deploy`** — packages code + requirements, uploads to S3, deploys to AgentCore Runtime

**Note:** `requirements.txt` should only contain runtime deps (fastmcp, mcp, uvicorn, etc). Do NOT include boto3 or the starter toolkit — they bloat the package beyond the 750MB limit.

In [ ]:
# Build the Cognito JWT authorizer config
# AgentCore uses this to validate incoming JWT tokens
# discoveryUrl: OpenID Connect discovery endpoint for token validation
# allowedClients: only tokens from this client ID are accepted
discovery_url = cognito_config['discovery_url'].replace('/jwks.json', '/openid-configuration')
auth_json = json.dumps({
    'customJWTAuthorizer': {
        'discoveryUrl': discovery_url,
        'allowedClients': [cognito_config['client_id']]
    }
})

# Configure the agent
# -e  : entrypoint Python file
# -n  : agent name (must be unique in your account)
# -p  : protocol (MCP)
# -dt : deployment type (direct_code_deploy = no Docker)
# -rt : Python runtime version
# -er : IAM execution role ARN
# -ac : authorizer config (Cognito JWT)
# -do : disable OpenTelemetry
# -dm : disable memory
# -ni : non-interactive (no prompts)
print('=== Configuring AgentCore ===\n')
!agentcore configure \
  -e hr_mcp_server.py \
  -n hr_mcp_server_v3 \
  -p MCP \
  -dt direct_code_deploy \
  -rt PYTHON_3_12 \
  -er {execution_role_arn} \
  -r {AWS_REGION} \
  -ac '{auth_json}' \
  -do \
  -dm \
  -ni

print('\nWaiting 30s for IAM role propagation...')
time.sleep(30)

# Deploy: zips code + requirements.txt, uploads to S3, creates AgentCore runtime
# --auto-update-on-conflict: if agent already exists, update it instead of failing
print('\n=== Deploying to AgentCore Runtime ===\n')
!agentcore deploy --auto-update-on-conflict

---
## Step 5: Get Agent ARN and MCP Endpoint URL

After deployment, the agent ARN is written back to `.bedrock_agentcore.yaml`.

The MCP endpoint URL format:
```
https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{url-encoded-arn}/invocations?qualifier=DEFAULT
```

In [ ]:
with open('.bedrock_agentcore.yaml', 'r') as f:
    deploy_config = yaml.safe_load(f)

agent_name = deploy_config['default_agent']
agent_arn = deploy_config['agents'][agent_name].get('bedrock_agentcore', {}).get('agent_arn', '')

if not agent_arn:
    print('❌ Agent ARN not found — check Step 4 output for errors.')
else:
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    MCP_ENDPOINT = f'https://bedrock-agentcore.{AWS_REGION}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT'
    print(f'✓ Agent ARN:    {agent_arn}')
    print(f'✓ MCP Endpoint: {MCP_ENDPOINT}')

---
## Step 6: Store Credentials

Stores the Agent ARN in SSM Parameter Store and Cognito credentials in Secrets Manager.
The test client scripts (`my_mcp_client_remote.py`, `invoke_mcp_tools.py`) retrieve these automatically.

In [ ]:
ssm = boto3.client('ssm', region_name=AWS_REGION)
ssm.put_parameter(Name='/hr_mcp_server/runtime/agent_arn', Value=agent_arn, Type='String', Overwrite=True)
print('✓ Agent ARN stored in SSM Parameter Store')

sm = boto3.client('secretsmanager', region_name=AWS_REGION)
secret = json.dumps({
    'bearer_token': cognito_config['bearer_token'],
    'refresh_token': cognito_config['refresh_token'],
    'client_id': cognito_config['client_id'],
    'client_secret': cognito_config['client_secret'],
    'pool_id': cognito_config['pool_id']
})
try:
    sm.create_secret(Name='hr_mcp_server/cognito/credentials', SecretString=secret)
except sm.exceptions.ResourceExistsException:
    sm.update_secret(SecretId='hr_mcp_server/cognito/credentials', SecretString=secret)
print('✓ Cognito credentials stored in Secrets Manager')

---
## Step 7: Test — List Available Tools

Connects to the deployed MCP server using the bearer token and lists all registered tools.

In [ ]:
import asyncio
from datetime import timedelta
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def list_tools():
    headers = {'authorization': f"Bearer {cognito_config['bearer_token']}"}
    async with streamablehttp_client(
        MCP_ENDPOINT, headers, timeout=timedelta(seconds=120), terminate_on_close=False
    ) as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            result = await session.list_tools()
            print('📋 Available HR Tools:')
            print('=' * 50)
            for tool in result.tools:
                print(f'🔧 {tool.name}: {tool.description[:80]}')
            print(f'\n✅ Found {len(result.tools)} tools')

await list_tools()

---
## Step 8: Test — Invoke All 5 HR Tools

Calls each tool with sample data for employees EMP001/EMP002 (defined in `hr_mcp_server.py`).

In [ ]:
async def test_all_tools():
    headers = {'authorization': f"Bearer {cognito_config['bearer_token']}"}
    async with streamablehttp_client(
        MCP_ENDPOINT, headers, timeout=timedelta(seconds=120), terminate_on_close=False
    ) as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tests = [
                ('get_employee_info',      {'employee_id': 'EMP001'}),
                ('check_leave_balance',    {'employee_id': 'EMP001'}),
                ('create_leave_request',   {'employee_id': 'EMP001', 'start_date': '2026-03-01', 'end_date': '2026-03-05', 'leave_type': 'vacation'}),
                ('update_employee_record', {'employee_id': 'EMP001', 'field': 'email', 'value': 'alice.new@company.com'}),
                ('create_support_ticket',  {'employee_id': 'EMP001', 'category': 'IT', 'description': 'Laptop cannot connect to VPN'}),
            ]
            for i, (name, args) in enumerate(tests, 1):
                print(f'\n{"=" * 50}')
                print(f'TEST {i}: {name}')
                print('=' * 50)
                try:
                    result = await session.call_tool(name=name, arguments=args)
                    print(f'✅ {result.content[0].text}')
                except Exception as e:
                    print(f'❌ Error: {e}')
            print(f'\n🎉 All tests completed!')

await test_all_tools()

---
## Step 9: 📋 Connection Details for Amazon Quick MCP Client

Copy these 4 values into the Amazon Quick MCP Client interface:

| Field | Where it comes from |
|-------|--------------------|
| **MCP Server URL** | AgentCore Runtime endpoint (from Step 5) |
| **Client ID** | Cognito app client (from Step 2) |
| **Client Secret** | Cognito app client secret (from Step 2) |
| **Token URL** | Cognito domain OAuth2 endpoint (from Step 2) |

**How auth works:**
1. Quick client sends `client_id` + `client_secret` to the Token URL (`client_credentials` grant)
2. Cognito returns a JWT access token with scope `hr-mcp/access`
3. Quick client sends the JWT in the `Authorization: Bearer` header to the MCP Server URL
4. AgentCore validates the JWT against the Cognito discovery URL
5. If valid, the request reaches your HR MCP server

In [ ]:
print('=' * 60)
print('📋 MCP SERVER CONNECTION DETAILS')
print('   Copy these into the Amazon Quick MCP Client')
print('=' * 60)
print()
print(f'MCP Server URL:   {MCP_ENDPOINT}')
print(f'Client ID:        {cognito_config["client_id"]}')
print(f'Client Secret:    {cognito_config["client_secret"]}')
print(f'Token URL:        {cognito_config["token_url"]}')
print()
print(f'Agent ARN:        {agent_arn}')
print(f'Pool ID:          {cognito_config["pool_id"]}')
print(f'Region:           {AWS_REGION}')
print()
print('=' * 60)
print()
print('💡 Sample prompts to try in Amazon Quick:')
print('   - "What is the leave balance for EMP001?"')
print('   - "Create a vacation request for EMP001 from March 1-5"')
print('   - "Show me employee info for EMP002"')
print('   - "Create an IT support ticket for EMP001 about VPN issues"')
print('   - "Update the email for EMP001 to alice.new@company.com"')

---
## ✅ Workshop Complete!

### What you built
- **Cognito** — User Pool + domain + resource server + scope + app client with secret
- **IAM Role** — Execution role with AgentCore trust policy
- **AgentCore Runtime** — HR MCP server deployed via direct code deploy (no Docker)
- **5 HR Tools** — get_employee_info, check_leave_balance, create_leave_request, update_employee_record, create_support_ticket
- **Amazon Q** — Connected via OAuth client_credentials flow

### Key concepts
- **Direct code deploy** — zip your Python code + requirements, upload to S3, deploy. No Docker, no ECR, no Dockerfile.
- **JWT auth flow** — Quick client gets a token from Cognito using client_credentials, sends it to AgentCore, AgentCore validates it.
- **Cognito domain** — required for the `/oauth2/token` endpoint to exist.
- **Resource server + scope** — required for `client_credentials` grant to work.
- **`requirements.txt`** — only include runtime deps. boto3 and the starter toolkit are NOT needed at runtime and will bloat the package past the 750MB limit.

### Cleanup
When done, run the cell below to remove all resources.

In [ ]:
# OPTIONAL: Cleanup all resources
# Uncomment and run when you're done with the workshop

# !agentcore destroy --force
# iam.delete_role(RoleName='AgentCore-HR-MCP-ExecutionRole')
# cog = boto3.client('cognito-idp', region_name=AWS_REGION)
# cog.delete_user_pool_domain(UserPoolId=cognito_config['pool_id'], Domain=cognito_config['token_url'].split('//')[1].split('.')[0])
# cog.delete_user_pool(UserPoolId=cognito_config['pool_id'])
# ssm.delete_parameter(Name='/hr_mcp_server/runtime/agent_arn')
# sm.delete_secret(SecretId='hr_mcp_server/cognito/credentials', ForceDeleteWithoutRecovery=True)
# print('✅ All resources cleaned up')